# Data Mining Semester Project - GBM Model
## Group 10 - Yoo Na Cha, Nupur Neti, Luan Nquyen, Maggie Wang



### Data preparation for GBM Model

In [0]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.targetencoder import TargetEncoder
from h2o.estimators.gbm import H2OGradientBoostingEstimator 
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
SEED = 12345

In [0]:

from h2o.estimators.random_forest import H2ORandomForestEstimator

In [0]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,2 mins 13 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.2
H2O cluster version age:,22 days
H2O cluster name:,H2O_from_python_yoona_38ymxb
H2O cluster total nodes:,1
H2O cluster free memory:,3.501 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [0]:
col_types = {'srch_adults_cnt':'enum',
             'srch_children_cnt':'enum',
             'srch_rm_cnt':'enum',
             'length_stay':'enum',
             'book_advance_day':'enum',
             'cnt':'enum',
             'hotel_location_id_te':'enum',
             'user_location_id_te':'enum',
             'srch_destination_id_te':'enum',
             'is_package':'categorical',
             'srch_destination_type_id':'categorical',
             'channel':'categorical'}

train = h2o.import_file(path = 'train_encoded.csv',col_types=col_types)
valid = h2o.import_file(path = 'valid_encoded.csv',col_types=col_types)
test = h2o.import_file(path = 'test_encoded.csv',col_types=col_types)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
train = train.drop(['user_location_id',
                    'hotel_location_id',
                    'user_id',
                    'srch_destination_id',
                    'srch_ci',
                    'srch_co',
                    'date_time',
                    'is_booking',
                    'srch_ci_dayofweek',
                    'srch_dayofweek',
                    'srch_ci_month'],axis=1)

In [0]:
valid = valid.drop(['user_location_id',
                    'hotel_location_id',
                    'user_id',
                    'srch_destination_id',
                    'srch_ci',
                    'srch_co',
                    'date_time',
                    'is_booking',
                    'srch_ci_dayofweek',
                    'srch_dayofweek',
                    'srch_ci_month'],axis=1)

In [0]:
test = test.drop(['user_location_id',
                  'hotel_location_id',
                  'user_id',
                  'srch_destination_id',
                  'srch_ci',
                  'srch_co',
                  'date_time',
                  'is_booking',
                  'srch_ci_dayofweek',
                  'srch_dayofweek',
                  'srch_ci_month'],axis=1)

In [0]:
train.describe()

Rows:562167
Cols:13




,hotel_cluster,srch_destination_type_id,channel,is_package,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,length_stay,book_advance_day,cnt,hotel_location_id_te,user_location_id_te,srch_destination_id_te
type,int,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum
mins,0.0,,,,,,,,,,,,
mean,49.9505378295062,,,,,,,,,,,,
maxs,99.0,,,,,,,,,,,,
sigma,29.14516232096619,,,,,,,,,,,,
zeros,6615,,,,,,,,,,,,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0
0,7.0,3,9,0,2,0,1,4,219,1,0.39142249740087065,0.20005846380452452,0.2927211096774654
1,28.0,3,0,0,2,3,1,1,1,3,0.3939858382481795,0.22015671516008933,0.17850919689072994
2,67.0,1,9,1,1,0,1,2,35,1,0.24909711122448575,0.17352099830454046,0.2529161138866801


In [0]:
valid.describe()

Rows:421626
Cols:13




,hotel_cluster,srch_destination_type_id,channel,is_package,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,length_stay,book_advance_day,cnt,hotel_location_id_te,user_location_id_te,srch_destination_id_te
type,int,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum
mins,0.0,,,,,,,,,,,,
mean,50.02742477930678,,,,,,,,,,,,
maxs,99.0,,,,,,,,,,,,
sigma,28.971762564929545,,,,,,,,,,,,
zeros,4839,,,,,,,,,,,,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0
0,30.0,4,5,0,2,0,1,1,38,1,0.20773286292518395,0.20374372739773056,0.20374372739773064
1,61.0,3,5,0,3,0,1,1,4,2,0.2738002329030685,0.20374372739773056,0.20374372739773064
2,82.0,4,2,0,2,0,3,1,23,3,0.15841573902719536,0.20374372739773056,0.20374372739773064


In [0]:
y = 'hotel_cluster'
x = [name for name in train.columns if name not in ['hotel_cluster']]

In [0]:
print(y)
print(x)

hotel_cluster
['srch_destination_type_id', 'channel', 'is_package', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt', 'length_stay', 'book_advance_day', 'cnt', 'hotel_location_id_te', 'user_location_id_te', 'srch_destination_id_te']


In [0]:
train[y] = train[y].asfactor()
valid[y] = valid[y].asfactor()
test[y] = test[y].asfactor()

In [0]:
# define random grid search parameters
hyper_parameters = {'ntrees':list(range(0, 500, 50)),
                    'max_depth':list(range(1, 20, 2)),
                    'sample_rate':[s/float(10) for s in range(1, 11)],
                    'col_sample_rate':[s/float(10) for s in range(1, 11)],'seed':12345}

# define search strategy
search_criteria = {'strategy':'RandomDiscrete',
                   'max_models':50,
                   'max_runtime_secs':600,'seed':12345}

# initialize grid search
gsearch = H2OGridSearch(H2OGradientBoostingEstimator,
                        hyper_params=hyper_parameters,
                        search_criteria=search_criteria)

# execute training w/ grid search
gsearch.train(x=x,
              y=y,
              training_frame=train,
              validation_frame=valid,seed=12345)

gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [0]:
# select best model
gbm_model = gsearch.get_grid()[0]

# print model information
gbm_model.show()

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  Grid_GBM_py_4_sid_934d_model_python_1557403520441_1_model_1


ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.9211931856419949
RMSE: 0.9597880941343224
LogLoss: 3.780199457728598
Mean Per-Class Error: 0.8939880529924307
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,Error,Rate
2062.0,57.0,8.0,3.0,2.0,22.0,0.0,0.0,2.0,9.0,4.0,3.0,1.0,3.0,0.0,0.0,2.0,8.0,12.0,2.0,4.0,8.0,6.0,0.0,0.0,18.0,382.0,0.0,1.0,1.0,14.0,26.0,0.0,1.0,408.0,1.0,6.0,1.0,0.0,0.0,1.0,134.0,1.0,0.0,12.0,6.0,45.0,0.0,5.0,1.0,8.0,9.0,169.0,1.0,17.0,9.0,266.0,7.0,12.0,11.0,2.0,19.0,9.0,181.0,277.0,1137.0,85.0,6.0,14.0,4.0,58.0,175.0,2.0,25.0,1.0,2.0,0.0,11.0,11.0,7.0,194.0,6.0,5.0,28.0,49.0,2.0,4.0,13.0,0.0,7.0,6.0,161.0,19.0,2.0,0.0,24.0,67.0,25.0,183.0,3.0,0.6882842,"4,553 / 6,615"
0.0,7069.0,1.0,3.0,0.0,1.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,17.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,37.0,1.0,0.0,0.0,257.0,6.0,0.0,9.0,1.0,0.0,6.0,0.0,0.0,21.0,7.0,14.0,2.0,0.0,2.0,1.0,1.0,4.0,0.0,16.0,0.0,0.0,0.0,1.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,99.0,0.0,0.0,6.0,4.0,0.0,0.0,1.0,0.0,22.0,6.0,1.0,35.0,0.0,3.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0808737,"622 / 7,691"
57.0,129.0,394.0,4.0,12.0,40.0,3.0,0.0,18.0,40.0,1.0,3.0,5.0,5.0,3.0,7.0,18.0,7.0,62.0,5.0,8.0,54.0,15.0,0.0,1.0,79.0,26.0,2.0,15.0,59.0,13.0,3.0,3.0,3.0,88.0,2.0,32.0,12.0,2.0,4.0,7.0,119.0,91.0,7.0,2.0,13.0,676.0,3.0,78.0,17.0,13.0,1.0,8.0,1.0,94.0,10.0,147.0,9.0,19.0,253.0,0.0,13.0,32.0,32.0,623.0,80.0,10.0,2.0,83.0,4.0,48.0,37.0,15.0,7.0,7.0,0.0,4.0,13.0,11.0,13.0,4.0,16.0,219.0,27.0,11.0,6.0,3.0,0.0,1.0,14.0,11.0,1323.0,6.0,4.0,5.0,606.0,18.0,86.0,278.0,23.0,0.9393566,"6,103 / 6,497"
118.0,135.0,30.0,198.0,11.0,26.0,2.0,7.0,12.0,15.0,4.0,3.0,5.0,5.0,2.0,1.0,10.0,5.0,27.0,5.0,26.0,27.0,6.0,0.0,1.0,45.0,125.0,7.0,8.0,24.0,38.0,6.0,3.0,1.0,80.0,1.0,19.0,10.0,9.0,3.0,3.0,192.0,41.0,3.0,28.0,14.0,248.0,6.0,92.0,2.0,5.0,9.0,39.0,8.0,38.0,10.0,247.0,15.0,16.0,77.0,4.0,28.0,36.0,94.0,291.0,219.0,22.0,14.0,17.0,6.0,71.0,47.0,4.0,18.0,3.0,0.0,3.0,4.0,19.0,16.0,47.0,27.0,119.0,29.0,20.0,11.0,4.0,4.0,0.0,33.0,6.0,550.0,31.0,1.0,3.0,168.0,34.0,33.0,157.0,10.0,0.9545455,"4,158 / 4,356"
125.0,129.0,21.0,3.0,377.0,17.0,1.0,3.0,4.0,21.0,3.0,5.0,3.0,2.0,1.0,5.0,10.0,8.0,43.0,8.0,3.0,55.0,1.0,2.0,2.0,57.0,47.0,0.0,13.0,11.0,5.0,2.0,6.0,1.0,22.0,0.0,9.0,6.0,1.0,5.0,5.0,164.0,38.0,3.0,2.0,5.0,141.0,4.0,118.0,17.0,12.0,3.0,4.0,2.0,37.0,10.0,355.0,6.0,6.0,175.0,1.0,4.0,14.0,20.0,154.0,61.0,1.0,0.0,46.0,1.0,105.0,10.0,11.0,9.0,0.0,0.0,2.0,13.0,2.0,10.0,52.0,11.0,62.0,58.0,3.0,4.0,0.0,1.0,1.0,13.0,3.0,1060.0,1.0,1.0,2.0,490.0,10.0,49.0,396.0,4.0,0.9219300,"4,452 / 4,829"
---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---
747.0,222.0,15.0,10.0,2.0,24.0,0.0,0.0,8.0,15.0,2.0,2.0,3.0,2.0,1.0,1.0,6.0,16.0,25.0,4.0,11.0,27.0,8.0,0.0,1.0,38.0,324.0,15.0,7.0,5.0,12.0,14.0,0.0,3.0,370.0,0.0,5.0,3.0,12.0,1.0,2.0,202.0,15.0,3.0,31.0,10.0,65.0,0.0,42.0,6.0,4.0,5.0,214.0,4.0,37.0,9.0,285.0,9.0,11.0,41.0,0.0,10.0,18.0,152.0,257.0,1340.0,113.0,9.0,11.0,6.0,94.0,167.0,6.0,16.0,2.0,0.0,2.0,22.0,14.0,9.0,73.0,3.0,16.0,43.0,53.0,4.0,13.0,18.0,2.0,15.0,5.0,396.0,123.0,2.0,2.0,57.0,324.0,32.0,219.0,6.0,0.9509834,"6,286 / 6,610"
233.0,333.0,41.0,3.0,17.0,50.0,5.0,1.0,18.0,24.0,10.0,7.0,5.0,2.0,1.0,2.0,5.0,12.0,17.0,2.0,5.0,67.0,31.0,1.0,8.0,139.0,51.0,0.0,15.0,34.0,19.0,13.0,0.0,2.0,86.0,4.0,27.0,9.0,1.0,5.0,9.0,260.0,14.0,0.0,16.0,26.0,567.0,1.0,36.0,3.0,4.0,0.0,33.0,1.0,14.0,24.0,696.0,8.0,14.0,89.0,3.0,13.0,18.0,64.0,1256.0,125.0,18.0,5.0,53.0,15.0,141.0,26.0,5.0,11.0,3.0,1.0,0.0,12.0,10.0,27.0,11.0,3.0,66.0,83.0,7.0,5.0,


See the whole table with table.as_data_frame()
Top-10 Hit Ratios: 


k,hit_ratio
1,0.1263308
2,0.2022993
3,0.2585246
4,0.3041605
5,0.3428963
6,0.3767244
7,0.4064380
8,0.4333125
9,0.4583105
10,0.4816683



ModelMetricsMultinomial: gbm
** Reported on validation data. **

MSE: 0.974372333674229
RMSE: 0.9871030005395734
LogLoss: 4.716571081920092
Mean Per-Class Error: 0.9856369607428462
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



...

k,hit_ratio
1,0.0257930
2,0.0471318
3,0.0676713
4,0.0875966
5,0.1058758
6,0.1226087
7,0.1387272
8,0.1543548
9,0.1695792
10,0.1842178


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
,2019-05-09 08:26:53,2.098 sec,0.0,0.9900000,4.6051702,0.9879840,0.9900000,4.6051702,0.9881910
,2019-05-09 08:27:52,1 min 1.098 sec,7.0,0.9781444,4.1423796,0.9087015,0.9889254,4.5609018,0.9801696
,2019-05-09 08:31:28,4 min 37.700 sec,25.0,0.9671633,3.9104018,0.8911480,0.9876259,4.6349531,0.9744086
,2019-05-09 08:36:58,10 min 7.202 sec,53.0,0.9597881,3.7801995,0.8736692,0.9871030,4.7165711,0.9742070


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
hotel_location_id_te,31383.4062500,1.0,0.4414917
book_advance_day,19558.6855469,0.6232174,0.2751453
srch_destination_id_te,10818.4296875,0.3447181,0.1521902
cnt,2253.2192383,0.0717965,0.0316976
srch_destination_type_id,2180.2419434,0.0694712,0.0306709
length_stay,1802.5336914,0.0574359,0.0253575
user_location_id_te,1172.0723877,0.0373469,0.0164883
is_package,989.9347534,0.0315433,0.0139261
srch_children_cnt,501.0410156,0.0159652,0.0070485
srch_adults_cnt,191.0111389,0.0060864,0.0026871


In [0]:
gbm_model.model_performance(test_data=test)


ModelMetricsMultinomial: gbm
** Reported on test data. **

MSE: 0.9748379067663786
RMSE: 0.9873388003954765
LogLoss: 4.7445338090852385
Mean Per-Class Error: 0.9853108348831416
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,Error,Rate
1.0,0.0,2.0,29.0,1.0,32.0,2.0,0.0,34.0,406.0,27.0,4.0,3.0,9.0,0.0,0.0,0.0,7.0,36.0,3.0,10.0,165.0,13.0,3.0,0.0,9.0,0.0,0.0,33.0,0.0,11.0,0.0,0.0,3.0,1.0,2.0,2.0,51.0,3.0,1.0,14.0,1374.0,0.0,1.0,2.0,0.0,2.0,3.0,63.0,1.0,20.0,2.0,0.0,10.0,0.0,36.0,0.0,10.0,51.0,0.0,7.0,3.0,18.0,1.0,598.0,0.0,0.0,4.0,66.0,4.0,115.0,0.0,10.0,2.0,0.0,0.0,2.0,19.0,9.0,0.0,0.0,1.0,4.0,17.0,0.0,6.0,6.0,0.0,0.0,21.0,11.0,343.0,0.0,7.0,0.0,20.0,2.0,438.0,869.0,26.0,0.9998047,"5,120 / 5,121"
0.0,6.0,2.0,6.0,1.0,30.0,0.0,2.0,12.0,465.0,28.0,0.0,3.0,3.0,1.0,1.0,0.0,1.0,10.0,10.0,0.0,284.0,14.0,1.0,0.0,2.0,2.0,0.0,24.0,0.0,14.0,0.0,1.0,5.0,0.0,1.0,2.0,52.0,1.0,0.0,18.0,1528.0,0.0,1.0,0.0,0.0,0.0,1.0,47.0,1.0,49.0,4.0,0.0,1.0,0.0,36.0,1.0,7.0,34.0,0.0,3.0,5.0,9.0,0.0,721.0,0.0,0.0,0.0,66.0,6.0,127.0,0.0,9.0,0.0,0.0,1.0,2.0,7.0,5.0,0.0,0.0,1.0,5.0,4.0,0.0,4.0,4.0,0.0,0.0,18.0,7.0,274.0,0.0,3.0,0.0,46.0,0.0,552.0,855.0,36.0,0.9989055,"5,476 / 5,482"
0.0,5.0,4.0,6.0,8.0,42.0,5.0,1.0,24.0,388.0,31.0,1.0,4.0,14.0,0.0,0.0,0.0,2.0,10.0,8.0,3.0,381.0,12.0,3.0,0.0,3.0,1.0,0.0,47.0,0.0,14.0,0.0,2.0,1.0,9.0,1.0,1.0,47.0,3.0,1.0,23.0,1398.0,2.0,3.0,0.0,0.0,2.0,0.0,68.0,1.0,42.0,0.0,0.0,3.0,0.0,30.0,5.0,10.0,50.0,0.0,2.0,1.0,26.0,0.0,633.0,0.0,0.0,5.0,60.0,6.0,120.0,2.0,6.0,0.0,0.0,1.0,1.0,26.0,8.0,1.0,0.0,0.0,3.0,9.0,0.0,6.0,4.0,0.0,0.0,21.0,9.0,461.0,1.0,2.0,1.0,50.0,2.0,458.0,669.0,35.0,0.9992521,"5,344 / 5,348"
0.0,0.0,0.0,8.0,0.0,11.0,1.0,1.0,6.0,91.0,7.0,0.0,3.0,6.0,0.0,0.0,0.0,1.0,7.0,2.0,4.0,115.0,1.0,2.0,0.0,2.0,0.0,0.0,11.0,0.0,4.0,0.0,0.0,4.0,1.0,1.0,0.0,18.0,3.0,0.0,16.0,500.0,1.0,0.0,2.0,0.0,4.0,1.0,23.0,1.0,11.0,1.0,0.0,0.0,0.0,5.0,0.0,9.0,14.0,0.0,6.0,0.0,7.0,0.0,152.0,0.0,0.0,3.0,17.0,3.0,56.0,0.0,2.0,0.0,0.0,0.0,2.0,4.0,2.0,0.0,3.0,0.0,1.0,5.0,0.0,4.0,3.0,0.0,0.0,29.0,2.0,161.0,0.0,3.0,0.0,13.0,0.0,116.0,180.0,12.0,0.9952494,"1,676 / 1,684"
0.0,0.0,0.0,5.0,4.0,15.0,2.0,1.0,10.0,159.0,17.0,0.0,1.0,15.0,0.0,1.0,2.0,2.0,6.0,8.0,1.0,366.0,4.0,2.0,0.0,1.0,3.0,0.0,18.0,1.0,9.0,0.0,0.0,6.0,0.0,0.0,1.0,30.0,1.0,3.0,34.0,1139.0,0.0,1.0,0.0,0.0,0.0,0.0,97.0,0.0,84.0,1.0,0.0,3.0,0.0,25.0,2.0,2.0,21.0,1.0,4.0,0.0,9.0,0.0,253.0,0.0,0.0,3.0,29.0,2.0,91.0,0.0,7.0,0.0,0.0,1.0,0.0,77.0,3.0,0.0,0.0,2.0,1.0,6.0,0.0,2.0,0.0,0.0,0.0,12.0,1.0,475.0,0.0,0.0,0.0,33.0,0.0,246.0,379.0,22.0,0.9989367,"3,758 / 3,762"
---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---
0.0,0.0,2.0,9.0,1.0,22.0,1.0,1.0,20.0,343.0,34.0,1.0,3.0,8.0,0.0,1.0,0.0,4.0,21.0,2.0,2.0,147.0,20.0,3.0,0.0,2.0,3.0,2.0,38.0,1.0,8.0,0.0,0.0,2.0,0.0,2.0,6.0,40.0,4.0,0.0,24.0,1207.0,1.0,0.0,7.0,0.0,2.0,0.0,48.0,1.0,22.0,2.0,0.0,8.0,0.0,33.0,0.0,7.0,47.0,1.0,4.0,2.0,20.0,0.0,530.0,0.0,0.0,3.0,52.0,5.0,92.0,2.0,12.0,4.0,0.0,1.0,1.0,15.0,4.0,0.0,28.0,1.0,1.0,19.0,0.0,4.0,3.0,0.0,0.0,11.0,10.0,384.0,2.0,5.0,0.0,22.0,2.0,372.0,682.0,17.0,0.9995529,"4,471 / 4,473"
0.0,10.0,1.0,10.0,3.0,40.0,1.0,1.0,32.0,373.0,32.0,5.0,3.0,6.0,1.0,0.0,1.0,2.0,16.0,8.0,1.0,283.0,30.0,7.0,0.0,6.0,0.0,1.0,23.0,0.0,13.0,0.0,0.0,5.0,64.0,0.0,2.0,39.0,1.0,1.0,17.0,1413.0,2.0,1.0,2.0,0.0,3.0,0.0,86.0,0.0,33.0,4.0,0.0,4.0,2.0,25.0,12.0,7.0,49.0,1.0,4.0,5.0,14.0,0.0,804.0,0.0,0.0,2.0,42.0,5.0,118.0,22.0,6.0,4.0,0.0,0.0,2.0,35.0,6.0,4.0,3.0,1.0,1.0,8.0,0.0,4.0,1.0,0.0,0.0,23.0,12.0,369.0,1.0,2.0,1.0,31.0,6.0,538.0,699.0,25.0,0.9018248,"4,942 / 5,480"
0.0,0.0,1.0,4.0,1.0,45.0,7.0,1.0,28.0,467.0,39.0,7.0,3.0


See the whole table with table.as_data_frame()
Top-10 Hit Ratios: 


k,hit_ratio
1,0.0241660
2,0.0449925
3,0.0655201
4,0.0844635
5,0.1021782
6,0.1185031
7,0.1341473
8,0.1497678
9,0.1644633
10,0.1785919


In [0]:
gbm_model.